In [1]:
import sys
sys.path.append('../')

In [2]:
!pip install sympy

Defaulting to user installation because normal site-packages is not writeable


In [6]:
import math
import sympy
from sympy import poly
import numpy
import dimod
from sympy import symbols
from copy import deepcopy
import neal

In [7]:
HUBO_Mol="./N_Rot_2/HUBO_20A12Rot_27#s2"

In [8]:
#numer of rotatable bond considered
num_rots=int(HUBO_Mol[HUBO_Mol.find("s")+1])

#Initialize variables
if num_rots==2:
    a0, a1, a2, a3  = symbols('a0, a1, a2, a3')
if num_rots==4:
    a0, a1, a2, a3, a4, a5, a6, a7  = symbols('a0, a1, a2, a3, a4, a5, a6, a7')
if num_rots==5:
    a0, a1, a2, a3, a4, a5, a6, a7, a8, a9  = symbols('a0, a1, a2, a3, a4, a5, a6, a7, a8, a9')
if num_rots==6:
    a0, a1, a2, a3, a4, a5, a6, a7, a8, a9, a10, a11  = symbols('a0, a1, a2, a3, a4, a5, a6, a7, a8, a9, a10, a11')
if num_rots==8:
    a0, a1, a2, a3, a4, a5, a6, a7, a8, a9, a10, a11, a12, a13, a14, a15  = symbols('a0, a1, a2, a3, a4, a5, a6, a7, a8, a9, a10, a11, a12, a13, a14, a15')

In [9]:
FILE= HUBO_Mol + ".txt"
total_FILE= HUBO_Mol + '_total.txt'

read_total = open(total_FILE, 'r').read()
total=eval(read_total)

In [10]:
 #This is used to check the maximum coefficient appearing in Hubo_B
A_const=0
read_dictionary_B = open(FILE, 'r').read()
HUBO_B=eval(read_dictionary_B)

#We set the Hard constraint strength as the (maximum coefficient appearing in Hubo_B)*const.
#const was empirically selected to be 10
const=10
A_const=max(map(abs, list(HUBO_B.values())))*const

#read the final HUBO
read_dictionary= open(FILE, 'r').read()
HUBO=eval(read_dictionary)

In [11]:
print("Current size of the HUBO:",len(HUBO))

Current size of the HUBO: 137


In [12]:
def threshold_approx(h, val=1):
    d =h.copy()
    monoms = h.keys()
    for m in monoms:     
        temp = d[m]
        if (temp < 0.0):
            temp = -1.0 * temp
        if (temp <= (10.0 ** (val))):
            del d[m]
    return d

In [13]:
 #Coefficints with absolute value less than 10^{threshold} are deleted from the HUBO.
threshold=2

HUBO=threshold_approx(HUBO,threshold)

print("Size of the HUBO after threshold approximation:",len(HUBO))

Size of the HUBO after threshold approximation: 73


In [14]:
 #calculate the strength parameter needed by make_quadratic
max_hubo_value=max(map(abs, list(HUBO.values())))
strength=1.5*max_hubo_value
#generate the bqm
bqm = dimod.make_quadratic(HUBO, strength, dimod.BINARY)

In [15]:
sampler = neal.SimulatedAnnealingSampler()

sample_size=10

In [16]:
sampleset = sampler.sample(bqm, num_reads=sample_size)
print(sampleset)

   1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16       energy num_oc.
4  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  1 -2771.167969       1
2  1  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0 -2758.628906       1
9  0  0  1  0  0  0  0  0  0  0  0  1  0  0  0  0 -2756.808594       1
8  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1 -2753.707031       1
0  1  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0 -2751.902344       1
3  1  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0 -2724.152344       1
5  1  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0 -2719.228271       1
1  0  0  0  0  0  1  0  0  0  0  1  0  0  0  0  0 -2701.320312       1
7  0  0  0  0  0  0  0  1  0  0  0  0  1  0  0  0 -2690.697021       1
6  0  0  0  0  0  0  1  0  0  0  0  1  0  0  0  0 -2680.427734       1
['BINARY', 10 rows, 10 samples, 16 variables]


In [17]:
new_answer=sampleset.record.sample

counter=0
solutions={}
occur_solutions=[]
occurrences={}

for s in range(len(new_answer)):
    bit_new_answer=new_answer[s]
    
    if num_rots==2:
    
        k1=tuple(i for i,x in enumerate(bit_new_answer[0:8]) if x == 1)
        k2=tuple(i for i,x in enumerate(bit_new_answer[8:16]) if x == 1)
        
        if len(k1)==1:
            if len(k2)==1:
                         
                occur_solutions.append((k1[0]+1, k2[0]+1))
                    
                c1=numpy.cos((numpy.pi/4.0)*(k1[0]+1))
                s1=numpy.sin((numpy.pi/4.0)*(k1[0]+1))
                c2=numpy.cos((numpy.pi/4.0)*(k2[0]+1))
                s2=numpy.sin((numpy.pi/4.0)*(k2[0]+1))
                
                minim=total(c1,s1,c2,s2)
                
                solutions[(k1[0]+1, k2[0]+1)]= minim
                counter=counter+1
                
    if num_rots==4:
        
        k1=tuple(i for i,x in enumerate(bit_new_answer[0:8]) if x == 1)
        k2=tuple(i for i,x in enumerate(bit_new_answer[8:16]) if x == 1)
        k3=tuple(i for i,x in enumerate(bit_new_answer[16:24]) if x == 1)
        k4=tuple(i for i,x in enumerate(bit_new_answer[24:32]) if x == 1)
        
        
        if len(k1)==1:
            if len(k2)==1:
                if len(k3)==1:
                    if len(k4)==1:
                        
                        occur_solutions.append((k1[0]+1, k2[0]+1, k3[0]+1, k4[0]+1))
            
                        c1=numpy.cos((numpy.pi/4.0)*(k1[0]+1))
                        s1=numpy.sin((numpy.pi/4.0)*(k1[0]+1))
                        c2=numpy.cos((numpy.pi/4.0)*(k2[0]+1))
                        s2=numpy.sin((numpy.pi/4.0)*(k2[0]+1))
                        c3=numpy.cos((numpy.pi/4.0)*(k3[0]+1))
                        s3=numpy.sin((numpy.pi/4.0)*(k3[0]+1))
                        c4=numpy.cos((numpy.pi/4.0)*(k4[0]+1))
                        s4=numpy.sin((numpy.pi/4.0)*(k4[0]+1))
                        
                        minim=total(c1,s1,c2,s2,c3,s3,c4,s4)
               
                        solutions[(k1[0]+1, k2[0]+1, k3[0]+1, k4[0]+1)]= minim
                        counter=counter+1
                        
    if num_rots==5:
        
        k1=tuple(i for i,x in enumerate(bit_new_answer[0:8]) if x == 1)
        k2=tuple(i for i,x in enumerate(bit_new_answer[8:16]) if x == 1)
        k3=tuple(i for i,x in enumerate(bit_new_answer[16:24]) if x == 1)
        k4=tuple(i for i,x in enumerate(bit_new_answer[24:32]) if x == 1)
        k5=tuple(i for i,x in enumerate(bit_new_answer[32:40]) if x == 1)
    
        
        if len(k1)==1:
            if len(k2)==1:
                if len(k3)==1:
                    if len(k4)==1:
                        if len(k5)==1:
                            
                            occur_solutions.append((k1[0]+1, k2[0]+1, k3[0]+1, k4[0]+1, k5[0]+1))
                            
                            c1=numpy.cos((numpy.pi/4.0)*(k1[0]+1))
                            s1=numpy.sin((numpy.pi/4.0)*(k1[0]+1))
                            c2=numpy.cos((numpy.pi/4.0)*(k2[0]+1))
                            s2=numpy.sin((numpy.pi/4.0)*(k2[0]+1))
                            c3=numpy.cos((numpy.pi/4.0)*(k3[0]+1))
                            s3=numpy.sin((numpy.pi/4.0)*(k3[0]+1))
                            c4=numpy.cos((numpy.pi/4.0)*(k4[0]+1))
                            s4=numpy.sin((numpy.pi/4.0)*(k4[0]+1))
                            c5=numpy.cos((numpy.pi/4.0)*(k5[0]+1))
                            s5=numpy.sin((numpy.pi/4.0)*(k5[0]+1))

                            minim=total(c1,s1,c2,s2,c3,s3,c4,s4,c5,s5)
                  
                            solutions[(k1[0]+1, k2[0]+1, k3[0]+1, k4[0]+1, k5[0]+1)]= minim
                            counter=counter+1
                            
    if num_rots==6:
        
        k1=tuple(i for i,x in enumerate(bit_new_answer[0:8]) if x == 1)
        k2=tuple(i for i,x in enumerate(bit_new_answer[8:16]) if x == 1)
        k3=tuple(i for i,x in enumerate(bit_new_answer[16:24]) if x == 1)
        k4=tuple(i for i,x in enumerate(bit_new_answer[24:32]) if x == 1)
        k5=tuple(i for i,x in enumerate(bit_new_answer[32:40]) if x == 1)
        k6=tuple(i for i,x in enumerate(bit_new_answer[40:48]) if x == 1)
        
        
        if len(k1)==1:
            if len(k2)==1:
                if len(k3)==1:
                    if len(k4)==1:
                        if len(k5)==1:
                            if len(k6)==1:
                        
                                occur_solutions.append((k1[0]+1, k2[0]+1, k3[0]+1, k4[0]+1, k5[0]+1, k6[0]+1))
            
                                c1=numpy.cos((numpy.pi/4.0)*(k1[0]+1))
                                s1=numpy.sin((numpy.pi/4.0)*(k1[0]+1))
                                c2=numpy.cos((numpy.pi/4.0)*(k2[0]+1))
                                s2=numpy.sin((numpy.pi/4.0)*(k2[0]+1))
                                c3=numpy.cos((numpy.pi/4.0)*(k3[0]+1))
                                s3=numpy.sin((numpy.pi/4.0)*(k3[0]+1))
                                c4=numpy.cos((numpy.pi/4.0)*(k4[0]+1))
                                s4=numpy.sin((numpy.pi/4.0)*(k4[0]+1))
                                c5=numpy.cos((numpy.pi/4.0)*(k5[0]+1))
                                s5=numpy.sin((numpy.pi/4.0)*(k5[0]+1))
                                c6=numpy.cos((numpy.pi/4.0)*(k6[0]+1))
                                s6=numpy.sin((numpy.pi/4.0)*(k6[0]+1))
                                
                                minim=total(c1,s1,c2,s2,c3,s3,c4,s4,c5,s5,c6,s6)
                      
                                solutions[(k1[0]+1, k2[0]+1, k3[0]+1, k4[0]+1, k5[0]+1, k6[0]+1)]= minim
                                counter=counter+1
                                
    if num_rots==8:
        
        k1=tuple(i for i,x in enumerate(bit_new_answer[0:8]) if x == 1)
        k2=tuple(i for i,x in enumerate(bit_new_answer[8:16]) if x == 1)
        k3=tuple(i for i,x in enumerate(bit_new_answer[16:24]) if x == 1)
        k4=tuple(i for i,x in enumerate(bit_new_answer[24:32]) if x == 1)
        k5=tuple(i for i,x in enumerate(bit_new_answer[32:40]) if x == 1)
        k6=tuple(i for i,x in enumerate(bit_new_answer[40:48]) if x == 1)
        k7=tuple(i for i,x in enumerate(bit_new_answer[48:56]) if x == 1)
        k8=tuple(i for i,x in enumerate(bit_new_answer[56:64]) if x == 1)
        
        
        
        if len(k1)==1:
            if len(k2)==1:
                if len(k3)==1:
                    if len(k4)==1:
                        if len(k5)==1:
                            if len(k6)==1:
                                if len(k7)==1:
                                    if len(k8)==1:
                        
                                        occur_solutions.append((k1[0]+1, k2[0]+1, k3[0]+1, k4[0]+1, k5[0]+1, k6[0]+1, k7[0]+1, k8[0]+1))
                    
                                        c1=numpy.cos((numpy.pi/4.0)*(k1[0]+1))
                                        s1=numpy.sin((numpy.pi/4.0)*(k1[0]+1))
                                        c2=numpy.cos((numpy.pi/4.0)*(k2[0]+1))
                                        s2=numpy.sin((numpy.pi/4.0)*(k2[0]+1))
                                        c3=numpy.cos((numpy.pi/4.0)*(k3[0]+1))
                                        s3=numpy.sin((numpy.pi/4.0)*(k3[0]+1))
                                        c4=numpy.cos((numpy.pi/4.0)*(k4[0]+1))
                                        s4=numpy.sin((numpy.pi/4.0)*(k4[0]+1))
                                        c5=numpy.cos((numpy.pi/4.0)*(k5[0]+1))
                                        s5=numpy.sin((numpy.pi/4.0)*(k5[0]+1))
                                        c6=numpy.cos((numpy.pi/4.0)*(k6[0]+1))
                                        s6=numpy.sin((numpy.pi/4.0)*(k6[0]+1))
                                        c7=numpy.cos((numpy.pi/4.0)*(k7[0]+1))
                                        s7=numpy.sin((numpy.pi/4.0)*(k7[0]+1))
                                        c8=numpy.cos((numpy.pi/4.0)*(k8[0]+1))
                                        s8=numpy.sin((numpy.pi/4.0)*(k8[0]+1))
                                        
                                        minim=total(c1,s1,c2,s2,c3,s3,c4,s4,c5,s5,c6,s6,c7,s7,c8,s8)
                                                                  
                                        solutions[(k1[0]+1, k2[0]+1, k3[0]+1, k4[0]+1, k5[0]+1, k6[0]+1, k7[0]+1, k8[0]+1)]= minim
    
                                        counter=counter+1
    

for sols in  list(set(occur_solutions)):
    occurrences[sols]=occur_solutions.count(sols)
    
Final_Solutions= sorted(solutions.items(), key=lambda x: x[1], reverse=True)

In [18]:
print( "Number of acceptable solutions found: ", counter) 

Number of acceptable solutions found:  10


In [19]:
print("All solutions found: ", Final_Solutions) 

All solutions found:  [((1, 8), 215.589769852800), ((3, 4), 214.265677153627), ((1, 1), 210.043271839780), ((4, 8), 192.387079292383), ((1, 2), 188.479711588632), ((1, 5), 155.357600716986), ((6, 3), 153.954258669861), ((1, 4), 149.811102703967), ((8, 5), 119.110256368122), ((7, 4), 110.558090270887)]


In [20]:
print( "Occurrences of each solution: ", occurrences) 

Occurrences of each solution:  {(7, 4): 1, (1, 2): 1, (3, 4): 1, (1, 5): 1, (1, 1): 1, (1, 8): 1, (1, 4): 1, (4, 8): 1, (6, 3): 1, (8, 5): 1}


In [ ]:
print( "Best Solution Found: ", Final_Solutions[0])
print( "Best Solution Occurences: ", occurrences[Final_Solutions[0][0]])